In [1]:
!git clone https://www.github.com/nvidia/apex
!cd ..
!pip install -v --no-cache-dir ./apex
!pip install simpletransformers

Cloning into 'apex'...
remote: Enumerating objects: 9513, done.
remote: Counting objects: 100% (584/584), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 9513 (delta 326), reused 358 (delta 176), pack-reused 8929
Receiving objects: 100% (9513/9513), 14.80 MiB | 8.39 MiB/s, done.
Resolving deltas: 100% (6437/6437), done.
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/py

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit
from simpletransformers.ner import NERModel
from seqeval.metrics import f1_score, classification_report

In [2]:
shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.3, random_state=2021)

In [3]:
df = pd.read_csv("disengagement_aggregated.csv")
df.words = df.words.astype('str')
df_post_train = pd.read_csv("post_train.csv")
df_post_test = pd.read_csv("post_test.csv")
df_post_train.words = df_post_train.words.astype(str)
df_post_train.labels = df_post_train.labels.astype(str)
df_post_test.words = df_post_test.words.astype(str)
df_post_test.labels = df_post_test.labels.astype(str)

## Post train

In [ ]:
start = time.time()
tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
model_args = {
    "output_dir": f"post_output/",
    'best_model_dir': f"post_output/best_model/",
    "manual_seed": 2020,
    "do_lower_case": True,
    "num_train_epochs": 8,
    "learning_rate": 5e-5,
    "max_seq_length": 128,

    "evaluate_during_training": True, 
    "evaluate_during_training_verbose": True,
    "evaluate_during_training_steps": 4450,
    "save_eval_checkpoints": False,

    "use_early_stopping": False,
    "early_stopping_delta": 0.001,
    "early_stopping_metric": "f1_score",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 3,

    'overwrite_output_dir': True,
}
labels = df_post_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
model = NERModel('electra', 'google/electra-base-discriminator', labels=tags, use_cuda = True, args=model_args)
model.train_model(df_post_train, eval_data = df_post_test)
result, model_outputs, predictions = model.eval_model(df_post_test)
end = time.time()
print(end - start)


Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.bias', 'class

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/99 [00:00<?, ?it/s]

790.2744443416595


In [ ]:
for _, test_index in shuffle_split.split(df.words, df.labels, groups=df.sentence_id):
  df_test = df.loc[test_index,:]
  tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
  model_args = {
    "output_dir": f"post_output/",
    'best_model_dir': f"post_output/best_model/",
    "manual_seed": 2020,
    "do_lower_case": True,
    "num_train_epochs": 8,
    "learning_rate": 5e-5,
    "max_seq_length": 128,

    "evaluate_during_training": True, 
    "evaluate_during_training_verbose": True,
    "evaluate_during_training_steps": 4450,
    "save_eval_checkpoints": False,

    "use_early_stopping": False,
    "early_stopping_delta": 0.001,
    "early_stopping_metric": "f1_score",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 3,

    'overwrite_output_dir': True,
}
  labels = df_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
  model = NERModel('electra', '/content/post_output/best_model/', labels=tags, use_cuda = True, args=model_args)
  result, model_outputs, predictions = model.eval_model(df_test)
  ind = []
  for i in range(len(labels)):
    if len(labels[i]) == len(predictions[i]):
      ind.append(i)
  labels = np.array(labels)[ind]
  predictions = np.array(predictions)[ind]
  print(classification_report(labels, predictions))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


              precision    recall  f1-score   support

           C       0.79      0.60      0.68       124
          CE       0.50      0.20      0.29        10
           E       0.50      0.49      0.49       100

   micro avg       0.64      0.54      0.58       234
   macro avg       0.60      0.43      0.49       234
weighted avg       0.65      0.54      0.59       234



  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.72      0.66      0.69       111
          CE       0.00      0.00      0.00         3
           E       0.54      0.57      0.55        93

   micro avg       0.63      0.61      0.62       207
   macro avg       0.42      0.41      0.41       207
weighted avg       0.63      0.61      0.62       207



  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.70      0.61      0.65       120
          CE       0.50      0.25      0.33         8
           E       0.50      0.50      0.50       101

   micro avg       0.60      0.55      0.57       229
   macro avg       0.57      0.45      0.50       229
weighted avg       0.60      0.55      0.57       229



In [ ]:
weighted_f1 = (0.59+0.60+0.53+0.62+0.57) / 5
print(f"weighted_f1 for post train is {weighted_f1}")
weighted_f1_std = np.std([0.59, 0.60, 0.53, 0.62, 0.57])
print(f"weighted_f1 for post train is {weighted_f1_std}")

weighted_f1 for post train is 0.582
weighted_f1 for post train is 0.0305941170815567


## Save best post train model for later use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/post_output/best_model/ /content/drive/MyDrive/cause_and_effect/best_model/

cp: cannot create directory '/content/drive/MyDrive/cause_and_effect/best_model/': No such file or directory


## post train + fine tuning

In [ ]:
start = time.time()
i = 0
for train_index, test_index in shuffle_split.split(df.words, df.labels, groups=df.sentence_id):
  df_train = df.loc[train_index,:]
  df_test = df.loc[test_index,:]
  tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
  model_args = {
      "output_dir": f"electra_all_outputs_{i}/",
      'best_model_dir': f"electra_all_outputs_{i}/best_model/",
      "do_lower_case": False,
      "manual_seed": 2020,
      "num_train_epochs": 8,
      "learning_rate": 5e-5,
      "max_seq_length": 128,

      "evaluate_during_training": True, 
      "evaluate_during_training_verbose": True,
      "evaluate_during_training_steps": 4450,
      "save_eval_checkpoints": False,

      "use_early_stopping": True,
      "early_stopping_delta": 0.01,
      "early_stopping_metric": "f1_score",
      "early_stopping_metric_minimize": False,
      "early_stopping_patience": 3,

      'overwrite_output_dir': True,
      'save_optimizer_and_scheduler': False,
      'save_model_every_epoch': False,
      'save_steps': -1,
  }
  labels_ = df_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
  model_ = NERModel('electra', '/content/post_output/best_model/', labels=tags, use_cuda = True, args=model_args)

  model_.train_model(df_train, eval_data = df_test)
  result_, model_outputs_, predictions_ = model_.eval_model(df_test)
  ind = []
  for i in range(len(labels_)):
    if len(labels_[i]) == len(predictions_[i]):
      ind.append(i)
  labels_ = np.array(labels_)[ind]
  predictions_ = np.array(predictions_)[ind]
  print(classification_report(labels_, predictions_))
  i += 1
end = time.time()
print(end - start)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.86      0.92      0.89       120
          CE       0.86      0.75      0.80         8
           E       0.84      0.92      0.88       101

   micro avg       0.85      0.91      0.88       229
   macro avg       0.85      0.86      0.85       229
weighted avg       0.85      0.91      0.88       229

662.0328624248505


In [ ]:
i = 0
for train_index, test_index in shuffle_split.split(df.words, df.labels, groups=df.sentence_id):
  df_train = df.loc[train_index,:]
  df_test = df.loc[test_index,:]
  tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
  model_args = {
      "output_dir": f"electra_all_outputs_{i}/",
      'best_model_dir': f"electra_all_outputs_{i}/best_model/",
      "do_lower_case": False,
      "manual_seed": 2020,
      "num_train_epochs": 8,
      "learning_rate": 5e-5,
      "max_seq_length": 128,

      "evaluate_during_training": True, 
      "evaluate_during_training_verbose": True,
      "evaluate_during_training_steps": 4450,
      "save_eval_checkpoints": False,

      "use_early_stopping": True,
      "early_stopping_delta": 0.01,
      "early_stopping_metric": "f1_score",
      "early_stopping_metric_minimize": False,
      "early_stopping_patience": 3,

      'overwrite_output_dir': True,
      'save_optimizer_and_scheduler': False,
      'save_model_every_epoch': False,
      'save_steps': -1,
  }
  labels_ = df_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
  model_ = NERModel('electra', f"electra_all_outputs_{i}/best_model/", labels=tags, use_cuda = True, args=model_args)
  result_, model_outputs_, predictions_ = model_.eval_model(df_test)
  ind = []
  for i in range(len(labels_)):
    if len(labels_[i]) == len(predictions_[i]):
      ind.append(i)
  labels_ = np.array(labels_)[ind]
  predictions_ = np.array(predictions_)[ind]
  print(classification_report(labels_, predictions_, 2))
  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


              precision    recall  f1-score   support

           C       0.87      0.91      0.89       124
          CE       0.80      0.80      0.80        10
           E       0.91      0.90      0.90       100

   micro avg       0.88      0.90      0.89       234
   macro avg       0.86      0.87      0.86       234
weighted avg       0.88      0.90      0.89       234



  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.95      0.94      0.95       111
          CE       1.00      1.00      1.00         3
           E       0.91      0.91      0.91        93

   micro avg       0.94      0.93      0.93       207
   macro avg       0.96      0.95      0.95       207
weighted avg       0.94      0.93      0.93       207



  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.89      0.91      0.90       120
          CE       0.86      0.75      0.80         8
           E       0.84      0.90      0.87       101

   micro avg       0.87      0.90      0.88       229
   macro avg       0.86      0.85      0.86       229
weighted avg       0.87      0.90      0.88       229



In [ ]:
weighted_f1 = (0.89 + 0.97 + 0.96 + 0.93 + 0.88) / 5
print(f"weighted_f1 for post train + fine tuning is {weighted_f1}")
weighted_f1_std = np.std([0.89, 0.97, 0.96, 0.93, 0.88])
print(f"weighted_f1 for post train + fine tuning is {weighted_f1_std}")

weighted_f1 for post train + fine tuning is 0.9259999999999999
weighted_f1 for post train + fine tuning is 0.03611094017053556


## fine tunig

In [4]:
start = time.time()
i = 0
for train_index, test_index in shuffle_split.split(df.words, df.labels, groups=df.sentence_id):
  df_train = df.loc[train_index,:]
  df_test = df.loc[test_index,:]
  tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
  model_args = {
      "output_dir": f"electra_outputs_{i}/",
      'best_model_dir': f"electra_outputs_{i}/best_model/",
      "do_lower_case": True,
      "manual_seed": 2020,
      "num_train_epochs": 8,
      "learning_rate": 5e-5,
      "max_seq_length": 128,

      "evaluate_during_training": True, 
      "evaluate_during_training_verbose": True,
      "evaluate_during_training_steps": 4450,
      "save_eval_checkpoints": False,

      "use_early_stopping": True,
      "early_stopping_delta": 0.01,
      "early_stopping_metric": "f1_score",
      "early_stopping_metric_minimize": False,
      "early_stopping_patience": 3,

      'overwrite_output_dir': True,
      'save_optimizer_and_scheduler': False,
      'save_model_every_epoch': False,
      'save_steps': -1,
  }
  labels_ = df_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
  model_ = NERModel('electra', 'google/electra-base-discriminator', labels=tags, use_cuda = True, args=model_args)
  #model_ = NERModel('roberta', '/content/post_output/best_model/', labels=tags, use_cuda = True, args=model_args)

  model_.train_model(df_train, eval_data = df_test)
  result_, model_outputs_, predictions_ = model_.eval_model(df_test)
  ind = []
  for i in range(len(labels_)):
    if len(labels_[i]) == len(predictions_[i]):
      ind.append(i)
  labels_ = np.array(labels_)[ind]
  predictions_ = np.array(predictions_)[ind]
  print(classification_report(labels_, predictions_, 2))
  i += 1
end = time.time()
print(end - start)

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.weight', 'cla

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.89      0.90      0.89       120
          CE       1.00      0.50      0.67         8
           E       0.80      0.85      0.83       101

   micro avg       0.85      0.86      0.86       229
   macro avg       0.90      0.75      0.80       229
weighted avg       0.85      0.86      0.86       229

608.9414076805115


In [5]:
i = 0
for train_index, test_index in shuffle_split.split(df.words, df.labels, groups=df.sentence_id):
  df_train = df.loc[train_index,:]
  df_test = df.loc[test_index,:]
  tags = ['O', 'B-C', 'I-C', 'B-E', 'I-E', 'B-CE', 'I-CE']
  model_args = {
      "output_dir": f"electra_outputs_{i}/",
      'best_model_dir': f"electra_outputs_{i}/best_model/",
      "do_lower_case": True,
      "manual_seed": 2020,
      "num_train_epochs": 8,
      "learning_rate": 5e-5,
      "max_seq_length": 128,

      "evaluate_during_training": True, 
      "evaluate_during_training_verbose": True,
      "evaluate_during_training_steps": 4450,
      "save_eval_checkpoints": False,

      "use_early_stopping": True,
      "early_stopping_delta": 0.01,
      "early_stopping_metric": "f1_score",
      "early_stopping_metric_minimize": False,
      "early_stopping_patience": 3,

      'overwrite_output_dir': True,
      'save_optimizer_and_scheduler': False,
      'save_model_every_epoch': False,
      'save_steps': -1,
  }
  labels_ = df_test.groupby(['sentence_id'])['labels'].apply(list).to_list()
  model_ = NERModel('electra', f"electra_outputs_{i}/best_model/", labels=tags, use_cuda = True, args=model_args)
  result_, model_outputs_, predictions_ = model_.eval_model(df_test)
  ind = []
  for i in range(len(labels_)):
    if len(labels_[i]) == len(predictions_[i]):
      ind.append(i)
  labels_ = np.array(labels_)[ind]
  predictions_ = np.array(predictions_)[ind]
  print(classification_report(labels_, predictions_, 2))
  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


              precision    recall  f1-score   support

           C       0.85      0.91      0.88       124
          CE       1.00      0.50      0.67        10
           E       0.91      0.86      0.88       100

   micro avg       0.88      0.87      0.87       234
   macro avg       0.92      0.76      0.81       234
weighted avg       0.88      0.87      0.87       234



  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.97      0.94      0.95       111
          CE       1.00      1.00      1.00         3
           E       0.89      0.91      0.90        93

   micro avg       0.93      0.93      0.93       207
   macro avg       0.95      0.95      0.95       207
weighted avg       0.93      0.93      0.93       207



  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           C       0.89      0.90      0.89       120
          CE       1.00      0.50      0.67         8
           E       0.80      0.85      0.83       101

   micro avg       0.85      0.86      0.86       229
   macro avg       0.90      0.75      0.80       229
weighted avg       0.85      0.86      0.86       229



In [ ]:
weighted_f1 = (0.86 + 0.92 + 0.93 + 0.90 + 0.86) / 5
print(f"weighted_f1 for post train + fine tuning is {weighted_f1}")
weighted_f1_std = np.std([0.86, 0.92, 0.93, 0.90, 0.86])
print(f"weighted_f1 for post train + fine tuning is {weighted_f1_std}")

weighted_f1 for post train + fine tuning is 0.8939999999999999
weighted_f1 for post train + fine tuning is 0.029393876913398165
